In [6]:
import ast

#base example of the node visitor
class v(ast.NodeVisitor):
   def generic_visit(self, node):
    print (type(node).__name__) #prints the type of value that is seen in the node
    ast.NodeVisitor.generic_visit(self, node) #generic visit to the node(iterates) (recursive)

x = v() #x is the function call
t = ast.parse('d[x] += v[y, x]') #example of a bit of code we are testing
x.visit(t) #call to generic_visit function

Module
AugAssign
Subscript
Name
Load
Name
Load
Store
Add
Subscript
Name
Load
Tuple
Name
Load
Name
Load
Load
Load


In [2]:
#example of node visitor that doesn't generic visit
#This means that the Load node isn't printed 

class w(v):
    def visit_Load(self, node): pass

y = w()
y.visit(t)

Module
AugAssign
Subscript
Name
Name
Store
Add
Subscript
Name
Tuple
Name
Name


In [3]:
'''This example is the same as the above code, but we are finding the actual names of
the values that are stored within the nodes themselves
'''
class z(v):
    def visit_Name(self, node): print ('Name:', node.id)

z().visit(t)

Module
AugAssign
Subscript
Name: d
Name: x
Store
Add
Subscript
Name: v
Tuple
Name: y
Name: x
Load
Load


In [4]:
'''This is a combination, so instead of not using generic visit, we use it and 
store all values that it finds into a vector (Something we need for comparing)
'''
class allnames(ast.NodeVisitor):
    def visit_Module(self, node):
        self.names = set()
        self.generic_visit(node)
        print (sorted(self.names))
    def visit_Name(self, node):
        self.names.add(node.id)

allnames().visit(t)

['d', 'v', 'x', 'y']


In [9]:
'''if we want an output that more low level for the computer, we can opt for this 
route. This could also help with future comparing'''

def str_node(node):
    if isinstance(node, ast.AST):
        fields = [(name, str_node(val)) for name, val in ast.iter_fields(node) if name not in ('left', 'right')]
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        return repr(node)
def ast_visit(node, level=0):
    print('  ' * level + str_node(node))
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value, level=level+1)


ast_visit(ast.parse('a * y'))

Module(body=[<ast.Expr object at 0x7f0c983870a0>], type_ignores=[])
  Expr(value=BinOp(op=Mult()))
    BinOp(op=Mult())
      Name(id='a', ctx=Load())
        Load()
      Mult()
      Name(id='y', ctx=Load())
        Load()


In [13]:
#attempting to parse a function in a different file
# Step 1: Read the source file
with open('test_code/battle_generator.py', 'r') as file:
    other_module_content = file.read()

# Step 2: Parse the content to get an AST
module_ast = ast.parse(other_module_content)

# Step 3: Navigate to the function
# for node in ast.walk(module_ast):
#     if isinstance(node, ast.FunctionDef) and node.name == 'format':
#         function_ast = node
#         break

# Now, function_ast is the AST of my_function
# print(ast.dump(function_ast, indent=4))


# class v(ast.NodeVisitor):
#    def generic_visit(self, node):
#     print (type(node).__name__) #prints the type of value that is seen in the node
#     ast.NodeVisitor.generic_visit(self, node) #generic visit to the node(iterates) (recursive)

# x = v() #x is the function call
# t = ast.parse('d[x] += v[y, x]') #example of a bit of code we are testing
# x.visit(t) #call to generic_visit function

def str_node(node):
    if isinstance(node, ast.AST):
        fields = [(name, str_node(val)) for name, val in ast.iter_fields(node) if name not in ('left', 'right')]
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        return repr(node)
def ast_visit(node, level=0):
    print('  ' * level + str_node(node))
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value, level=level+1)


ast_visit(module_ast)

Module(body=[<ast.Import object at 0x7f0c88e77fa0>, <ast.Import object at 0x7f0c88e77f40>, <ast.Import object at 0x7f0c88e77ee0>, <ast.Assign object at 0x7f0c88e77e80>, <ast.Assign object at 0x7f0c88e77d30>, <ast.Assign object at 0x7f0c88e77bb0>, <ast.FunctionDef object at 0x7f0c88e77a30>, <ast.Assign object at 0x7f0c88e76e00>, <ast.Assign object at 0x7f0c88e76d10>, <ast.Assign object at 0x7f0c88e76b90>, <ast.Assign object at 0x7f0c88e768c0>, <ast.Assign object at 0x7f0c88e76530>, <ast.For object at 0x7f0c88e76320>, <ast.Assign object at 0x7f0c88e74580>, <ast.With object at 0x7f0c88e74400>, <ast.Expr object at 0x7f0c88e74190>], type_ignores=[])
  Import(names=[<ast.alias object at 0x7f0c88e77f70>])
    alias(name='json', asname=None)
  Import(names=[<ast.alias object at 0x7f0c88e77f10>])
    alias(name='os', asname=None)
  Import(names=[<ast.alias object at 0x7f0c88e77eb0>])
    alias(name='string', asname=None)
  Assign(targets=[<ast.Name object at 0x7f0c88e77e50>], value=Call(func=At